In [1]:
import torch
from torch import nn
from d2l import torch as d2l

输入表示

In [2]:
'''
这段代码的主要功能是生成BERT模型的输入序列。
它会将单个文本或文本对转换成BERT输入序列，
并生成相应的片段索引。
'''
def get_tokens_and_segments(tokens_a, tokens_b=None):
    """
    获取输入序列的词元及其片段索引
    
    1. 单个文本输入
    如果只输入一个文本序列 `tokens_a`，
    函数会在序列的开头加上一个特殊词元 `<cls>`，
    在序列的结尾加上一个特殊词元 `<sep>`，并生成对应的片段索引。
    
    - `tokens` 是由特殊词元 `<cls>` 开头，
        接着是 `tokens_a` 的所有词元，最后加上 `<sep>`。

    """
    tokens = ['<cls>'] + tokens_a + ['<sep>']
    '''
    0和1分别标记片段A和B
    
    - `segments` 是一个全0的列表，长度与 `tokens` 相同，
        表示这些词元都属于片段A。
    '''
    segments = [0] * (len(tokens_a) + 2)
    '''
    2. 文本对输入
    
    如果输入两个文本序列 `tokens_a` 和 `tokens_b`，
    函数会将两个序列连接起来，中间和结尾加上 `<sep>`，
    并生成对应的片段索引。
    '''
    if tokens_b is not None:
        '''
        - `tokens` 是由特殊词元 `<cls>` 开头，
            接着是 `tokens_a` 的所有词元，
            然后是 `<sep>`，再接着是 `tokens_b` 的所有词元，
            最后再加上一个 `<sep>`。
        - `segments` 在 `tokens_a` 部分是全0的列表，
            在 `tokens_b` 部分是全1的列表，
            表示 `tokens_a` 属于片段A，`tokens_b` 属于片段B。
        '''
        tokens += tokens_b + ['<sep>']
        segments += [1] * (len(tokens_b) + 1)
    return tokens, segments

In [3]:
tokens_a = ['hello', ',', 'world', '!']
tokens_b = ['how', 'are', 'you', '?']
tokens, segments = get_tokens_and_segments(tokens_a, tokens_b)
print(tokens)    # ['<cls>', 'hello', ',', 'world', '!', '<sep>', 'how', 'are', 'you', '?', '<sep>']
print(segments)  # [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]


['<cls>', 'hello', ',', 'world', '!', '<sep>', 'how', 'are', 'you', '?', '<sep>']
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]


In [4]:
'''
BERT（Bidirectional Encoder Representations from Transformers）
是一种强大的自然语言处理模型，它使用了Transformer架构的一部分（编码器部分），
并增加了片段嵌入和可学习的位置嵌入。
'''
class BERTEncoder(nn.Module):
    '''
    - `vocab_size`：词汇表的大小，即模型可以处理的不同词元的数量。
    - `num_hiddens`：隐藏单元的数量，表示每个词元的嵌入向量的长度。
    - `norm_shape`：规范化层的形状。
    - `ffn_num_input` 和 `ffn_num_hiddens`：
        前馈神经网络的输入和隐藏单元数量。
    - `num_heads`：多头注意力机制中的头数。
    - `num_layers`：编码器层的数量。
    - `dropout`：Dropout概率，用于防止过拟合。
    - `max_len`：最大序列长度。
    - `key_size`, `query_size`, `value_size`：
        注意力机制的键、查询和值的维度。
    '''
    def __init__(
        self, 
        vocab_size,
        num_hiddens,
        norm_shape,
        ffn_num_input,
        ffn_num_hiddens,
        num_heads,
        num_layers,
        dropout,
        max_len=1000,
        key_size=768,
        query_size=768,
        value_size=768,
        **kwargs
    ):
        super(BERTEncoder, self).__init__(**kwargs)
        '''
        词元嵌入层：将词表中的索引转化为向量表示
        - `token_embedding`：词元嵌入层，用于将词元转换为嵌入向量。
        '''
        self.token_embedding = nn.Embedding(
            vocab_size, num_hiddens
        )
        '''
        片段嵌入层：区分不同的文本段落
        - `segment_embedding`：片段嵌入层，用于区分两个文本片段。
        '''
        self.segment_embedding = nn.Embedding(
            2, num_hiddens
        )
        '''
        编码器块的容器
        - `blks`：编码器块的顺序容器，包含多个编码器层。
        '''
        self.blks = nn.Sequential()
        '''
        添加多层编码器块
        '''
        for i in range(num_layers):
            self.blks.add_module(
                f'{i}',
                d2l.EncoderBlock(
                    key_size,
                    query_size,
                    value_size,
                    num_hiddens,
                    norm_shape,
                    ffn_num_input,
                    ffn_num_hiddens,
                    num_heads,
                    dropout,
                    True
                )
            )
        '''
        位置嵌入：用来表示输入序列中词元的位置，位置嵌入是可学习的
        在BERT中，位置嵌入是可学习的，因此我们创建一个足够长的位置嵌入参数
        
        - `pos_embedding`：
            可学习的位置嵌入参数，用于表示词元在序列中的位置。
        '''
        self.pos_embedding = nn.Parameter(
            torch.randn(
                1, max_len, num_hiddens
            )
        )
    '''
    - `tokens`：输入的词元序列，形状为 `(批量大小, 最大序列长度)`。
    - `segments`：片段索引，
        形状与 `tokens` 相同，用于区分不同的文本片段。
    - `valid_lens`：有效长度，用于处理不同长度的序列。
    '''
    def forward(self, tokens, segments, valid_lens):
        '''
        **嵌入层**：
        将词元嵌入和片段嵌入相加
        将词元和片段索引分别通过嵌入层，并将它们相加。
        '''
        X = self.token_embedding(tokens) + self.segment_embedding(segments)
        '''
        加上位置嵌入
        将位置嵌入添加到 `X` 中，表示词元在序列中的位置。
        '''
        X = X + self.pos_embedding.data[:, :X.shape[1], :]
        '''
        **编码器块**：
        通过每个编码器块进行处理
        每个块进行自注意力计算和前馈神经网络处理。

        '''
        for blk in self.blks:
            X = blk(X, valid_lens)
        '''
        **返回值**：
        - `X`：编码后的表示，形状为 
            `(批量大小, 最大序列长度, num_hiddens)`。

        '''
        return X
        

In [5]:
'''
vocab_size 词表大小为10000
'''
vocab_size, num_hiddens, ffn_num_hiddens, num_heads = 10000, 768, 1024,4
norm_shape, ffn_num_input, num_layers, dropout = [768], 768, 2, 0.2
encoder = BERTEncoder(
    vocab_size,
    num_hiddens,
    norm_shape,
    ffn_num_input,
    ffn_num_hiddens,
    num_heads,
    num_layers,
    dropout
)

In [6]:
'''
我们定义两个长度为8的输入序列，
每个序列中的词元是从词表中随机选择的索引。
片段标记用于区分输入的不同部分。

**生成随机输入**
- `tokens`：形状为 `(2, 8)` 的随机词元序列，
    每个词元是一个在 `[0, vocab_size)` 范围内的随机整数。
- `segments`：形状为 `(2, 8)` 的片段索引，
    第一个序列的前四个词元属于片段A（标记为0），
    后四个词元属于片段B（标记为1）。
'''

    
    

tokens = torch.randint(0, vocab_size, (2,8))
segments = torch.tensor(
    [[0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 1, 1, 1, 1, 1]]
)
'''
**编码输入数据**：
我们将输入的 `tokens` 和 `segments` 传入 BERT 编码器进行前向推断，
并输出编码结果。每个词元都会被表示为一个长度为 `num_hiddens` 的向量。
'''
encoded_X = encoder(
    tokens, segments, None
)
'''
输出的形状为 `[2, 8, 768]`，表示：
- 我们有2个输入序列
- 每个输入序列有8个词元
- 每个词元被表示为一个768维的向量


3. **输出形状**：
    - `encoded_X.shape`：显示编码结果的形状为 `(2, 8, 768)`，表示有2个序列，每个序列长度为8，每个词元被编码为长度为768的向量。


'''
encoded_X.shape

torch.Size([2, 8, 768])

- **词元嵌入层** 将词表中的索引转化为向量表示。
- **片段嵌入层** 用于区分不同的文本段落。
- **位置嵌入** 用于表示输入序列中词元的位置，并且是可学习的。
- **编码器块** 是由多个Transformer编码器块组成的，每个编码器块都会对输入数据进行处理。

通过这种方式，BERT能够处理单文本输入和文本对输入，并为每个词元生成一个高维的向量表示，这些表示可以用于下游的各种自然语言处理任务。

---
预训练任务

掩蔽语言模型

In [7]:
'''
掩蔽语言模型（Masked Language Modeling，MLM）任务的类`MaskLM`，
用于BERT预训练。
'''

class MaskLM(nn.Module):
    '''
    Bert的掩蔽语言模型任务
    
    - `vocab_size`：词表大小，即模型需要预测的不同词的数量。  
    - `num_hiddens`：隐藏层的大小。
    - `num_inputs`：输入的特征数量，默认为768。
    - `self.mlp`：定义了一个多层感知机（MLP），用于预测掩蔽词元。它包含：
        一个线性层，将输入特征数从`num_inputs`（768）变换到`num_hiddens`。
        一个ReLU激活函数。
        一个LayerNorm层，用于对隐藏层输出进行规范化。
        一个线性层，将隐藏层的输出变换到词表大小（`vocab_size`）。
    '''
    def __init__(
        self, 
        vocab_size,
        num_hiddens,
        num_inputs=768,
        **kwargs
    ):
        super(MaskLM, self).__init__(**kwargs)
        self.mlp = nn.Sequential(
            nn.Linear(num_inputs, num_hiddens),
            nn.ReLU(),
            nn.LayerNorm(num_hiddens),
            nn.Linear(num_hiddens, vocab_size)
        )
    '''
    - **输入**：
      - `X`：来自BERT编码器的输出，
          形状为（批量大小，序列长度，隐藏单元数）。
      - `pred_positions`：需要预测的掩蔽词元的位置，
          形状为（批量大小，预测位置数量）。
    '''
    def forward(self, X, pred_positions):
        '''
        - **过程**：
        - `num_pred_positions`：预测位置的数量。
        '''
        num_pred_positions = pred_positions.shape[1]
        '''
        - `pred_positions = pred_positions.reshape(-1)`：
          将预测位置展开为一维向量。
        '''
        pred_positions = pred_positions.reshape(-1)
        '''
        - `batch_size = X.shape[0]`：批量大小。
        '''
        batch_size = X.shape[0]
        '''
        - `batch_idx = torch.arange(0, batch_size)`：
          生成一个包含批量索引的张量。
        '''
        batch_idx = torch.arange(0, batch_size)
        '''
        - `batch_idx = torch.repeat_interleave(batch_idx, num_pred_positions)`：
          重复批量索引，使得每个批量索引对应每个预测位置。
        '''
        batch_idx = torch.repeat_interleave(
            batch_idx, num_pred_positions
        )
        '''
        - `masked_X = X[batch_idx, pred_positions]`：
          从`X`中选出需要预测的位置，
          得到的`masked_X`形状为（批量大小 * 预测位置数量，隐藏单元数）。
        '''
        masked_x = X[batch_idx, pred_positions]
        '''
        - `masked_X = masked_X.reshape((batch_size, num_pred_positions, -1))`：
          重新调整`masked_X`的形状，
          使其为（批量大小，预测位置数量，隐藏单元数）。
        '''
        masked_x = masked_x.reshape(
            (batch_size, num_pred_positions, -1)
        )
        '''
        - `mlm_Y_hat = self.mlp(masked_X)`：
          通过MLP计算掩蔽位置的预测结果。
        '''
        mlm_Y_hat = self.mlp(masked_x)
        '''
        - **输出**：
          - `mlm_Y_hat`：预测结果，
          形状为（批量大小，预测位置数量，词表大小）。
        '''
        return mlm_Y_hat

In [8]:
'''
实例化和前向推断

- **实例化**：创建`MaskLM`的实例，
    `vocab_size`和`num_hiddens`分别为词表大小和隐藏单元数。
'''
mlm = MaskLM(vocab_size, num_hiddens)
'''
- **定义预测位置**：
    `mlm_positions`表示每个输入序列中需要预测的词元位置。
'''
mlm_positions = torch.tensor(
    [
        [1, 5, 2],
        [6, 1, 5]
    ]
)
'''
- **前向推断**：通过调用`mlm`实例的`forward`方法，计算预测结果`mlm_Y_hat`。
    输出形状为（批量大小，预测位置数量，词表大小）。
'''
mlm_Y_hat = mlm(encoded_X, mlm_positions)
mlm_Y_hat.shape

torch.Size([2, 3, 10000])

In [9]:
'''
计算损失

- **定义真实标签**：`mlm_Y`是掩蔽位置的真实标签。
'''
mlm_Y = torch.tensor(
    [
        [7, 8, 9],
        [10, 20, 30]
    ]
)
'''
- **损失函数**：
    使用交叉熵损失函数计算预测结果和真实标签之间的差距。
  - `reduction='none'`：不进行损失的平均或求和操作。
'''
loss = nn.CrossEntropyLoss(reduction='none')
'''
- **计算损失**：
  - `mlm_Y_hat.reshape((-1, vocab_size))`：
      将预测结果调整为二维张量，形状为（批量大小 * 预测位置数量，词表大小）。
  - `mlm_Y.reshape(-1)`：
      将真实标签调整为一维张量，形状为（批量大小 * 预测位置数量）。
  - `mlm_l = loss(...)`：
      计算损失，输出形状为（批量大小 * 预测位置数量）。
'''
mlm_l = loss(
    mlm_Y_hat.reshape(
        (-1, vocab_size)
    ),
    mlm_Y.reshape(-1)
)
'''
BERT的掩蔽语言模型任务可以利用双向上下文信息来预测被掩蔽的词元，
从而使模型能够更好地理解和生成自然语言。
'''
mlm_l.shape

torch.Size([6])

---
下一句预测

这段代码实现并演示了BERT模型中的下一句预测（Next Sentence Prediction, NSP）任务。

In [15]:
class NextSentencePred(nn.Module):
    '''
    Bert的下一句预测任务
    '''
    def __init__(self, num_inputs, **kwargs):
        super(NextSentencePred, self).__init__(**kwargs)
        self.output = nn.Linear(num_inputs, 2)

    def forward(self, X):
        '''
        X的形状：（batchsize, num_hiddens）
        - `forward` 方法中，输入 `X` 的形状为 `(batch_size, num_hiddens)`。
        - 使用线性层 `self.output` 计算输出，
            返回形状为 `(batch_size, 2)` 的张量，
            每行对应一个输入序列的二分类结果。

        '''
        return self.output(X)

In [21]:
'''
准备输入数据
- 这里的 `encoded_X` 是由BERT编码器生成的表示，
    形状为 `(batch_size, seq_length, num_hiddens)`。
- 使用 `torch.flatten` 方法，
    将其展平为 `(batch_size, num_hiddens)` 的形状，以适应NSP的输入要求。
'''
encoded_X = torch.flatten(encoded_X, start_dim=1)
'''
创建NSP实例并进行前向推断
- 创建一个 `NextSentencePred` 的实例 `nsp`，
    其输入大小为 `encoded_X` 的最后一个维度大小（即 `num_hiddens`）。
- 对 `encoded_X` 进行前向推断，
    得到 `nsp_Y_hat`，其形状为 `(batch_size, 2)`。
'''
nsp = NextSentencePred(encoded_X.shape[-1])
nsp_Y_hat = nsp(encoded_X)
nsp_Y_hat.shape

torch.Size([2, 2])

In [23]:
'''
准备标签并计算损失

- `nsp_y` 是真实的标签，表示两个输入序列的下一句预测结果。
    `0` 表示下一句是连续的，`1` 表示下一句不是连续的。
- 使用交叉熵损失函数 `loss` 
    计算 `nsp_Y_hat` 和 `nsp_y` 之间的损失 `nsp_l`。
'''
nsp_y = torch.tensor([0, 1])
nsp_l = loss(nsp_Y_hat, nsp_y)
nsp_l.shape

torch.Size([2])

1. **定义NSP类**：实现一个简单的多层感知机（MLP）来进行二分类。
2. **准备输入数据**：将BERT编码器生成的表示展平，以适应NSP的输入要求。
3. **创建NSP实例并进行前向推断**：生成二分类结果。
4. **准备标签并计算损失**：使用真实标签计算预测结果的损失。

通过这段代码，我们实现并演示了如何使用BERT模型进行下一句预测任务。这一任务有助于模型理解句子之间的逻辑关系，从而提升其在自然语言处理任务中的表现。